In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import os
import time
import pandas as pd
from pandas import DataFrame
from datetime import datetime
import undetected_chromedriver as uc
from fake_useragent import UserAgent
import pickle
import gspread
from google.oauth2.service_account import Credentials

In [2]:
# Create a fake user-agent
ua = UserAgent()
fake_user_agent = ua.random  # Get a random user-agent

# Set up undetected-chromedriver
options = uc.ChromeOptions()
options.add_argument(f"user-agent={fake_user_agent}") 
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-cache")
options.add_argument("--aggressive-cache-discard")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-offline-load-stale-cache")
options.add_argument("--disk-cache-size=0")
# Uncomment if you want headless mode
# options.add_argument("--headless=new")

# Initialize undetected-chromedriver
driver = uc.Chrome(version_main=139, options=options)

# driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")
linkedin_username = "utkarsh.s@lawsikho.in"
linkedin_password = "Team123#"

# Define LinkedIn URL and cookies file
# linkedin_url = "https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin" 
linkedin_url = "https://www.linkedin.com/feed/" 
cookies_file = f"{linkedin_username}_cookies.pkl"

# Function to save cookies to a file
def save_cookies(driver, file_path):
    with open(file_path, "wb") as file:
        pickle.dump(driver.get_cookies(), file)

# Function to load cookies from a file
def load_cookies(driver, file_path):
    try:
        with open(file_path, "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)
    except (EOFError, pickle.UnpicklingError) as e:
        print(f"Error loading cookies: {e}. Deleting corrupted file.")
        os.remove(file_path)  # Remove the corrupted file

try:
    # Navigate to LinkedIn
    driver.get(linkedin_url)
    time.sleep(5)  # Wait for page to load

    # Check if cookies exist
    try:
        load_cookies(driver, cookies_file)
        print("Cookies loaded successfully.")
        
        driver.refresh()  # Refresh to apply cookies

        # enter_username = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,"//button[@class='member-profile__details']")))
        # enter_username.click()
    except FileNotFoundError:
        print("Cookies not found. Logging in manually.")

        enter_username = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,"//input[@id='username']")))
        enter_username.send_keys(linkedin_username)
        
        enter_password = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,"//input[@id='password']")))
        enter_password.send_keys(linkedin_password)
        
        enter_password.send_keys(Keys.RETURN)
        time.sleep(10)  # Wait for login to complete

        # Save cookies after successful login
        save_cookies(driver, cookies_file)
        print("Cookies saved successfully.")

finally:
    print("done")

Cookies loaded successfully.
done


In [6]:
search = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,"//input[@placeholder='Search']")))
search.send_keys("AfterQuery Experts")
search.send_keys(Keys.RETURN)

In [7]:
search = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,"//div[@class='GCGAvYKaRvwulIcqzIuxdlfUKudgKpVbO zlVNJAzocxpIgTRHjJgidFnSWtDViMcXCbFgw pt3 pb3 t-12 t-black--light']")))
search=search.text

print(search)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x128a8a3+63283]
	GetHandleVerifier [0x0x128a8e4+63348]
	(No symbol) [0x0x10c3e43]
	(No symbol) [0x0x110c8de]
	(No symbol) [0x0x110cc7b]
	(No symbol) [0x0x1154ef2]
	(No symbol) [0x0x1131464]
	(No symbol) [0x0x115271a]
	(No symbol) [0x0x1131216]
	(No symbol) [0x0x1100855]
	(No symbol) [0x0x11016f4]
	GetHandleVerifier [0x0x14fbb43+2623955]
	GetHandleVerifier [0x0x14f6daa+2604090]
	GetHandleVerifier [0x0x12b069a+218410]
	GetHandleVerifier [0x0x12a0ed8+154984]
	GetHandleVerifier [0x0x12a742d+180925]
	GetHandleVerifier [0x0x12922b8+94536]
	GetHandleVerifier [0x0x1292442+94930]
	GetHandleVerifier [0x0x127d5ea+9338]
	BaseThreadInitThunk [0x0x769afcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x772b82ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x772b827e+238]


In [8]:
search = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH,"//a[normalize-space()='AfterQuery Experts']")))
search=search.text
print(search)

AfterQuery Experts


In [9]:
try:
    search = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,"//a[normalize-space()='AfterQuery Experts']")))
    search=search.get_attribute("href")
    print(search)
except:
    print("No")

https://www.linkedin.com/company/afterquery-experts/


In [14]:
# https://www.linkedin.com/search/results/companies/?keywords=AfterQuery%20Experts

keyword = "Techmagnate SA"

keyword=keyword.replace(" ", "%20")
link = f"https://www.linkedin.com/search/results/companies/?keywords={keyword}"
link

'https://www.linkedin.com/search/results/companies/?keywords=Techmagnate%20SA'

In [20]:
def fetch_linkedin_link(filter, keyword):
    # Function to fetch LinkedIn company data
    keyword = keyword.lower()
    keyword = keyword.replace(" ", "%20")
    link = f"https://www.linkedin.com/search/results/{filter.lower()}/?keywords={keyword}"
    return link

fetch_linkedin_link("Hello", "AfterQuery Experts")

'https://www.linkedin.com/search/results/hello/?keywords=afterquery%20experts'